In [1]:
import torch
with torch.no_grad():
    torch.cuda.empty_cache()

In [2]:
import quanto
from transformers import AutoModelForCausalLM, AutoTokenizer
from lm_eval import evaluator
from lm_eval.models.huggingface import HFLM as eval_wrapper
from lm_eval.tasks import get_task_dict
from typing import Optional
import torch

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

2024-04-05:09:50:58,365 INFO     [modeling.py:965] We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
/scratch/user/vincent2013/myenv/lib/python3.10/site-packages/accelerate/utils/modeling.py:1341: UserWarning: Current model requires 1073750016 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [5]:
import lm_eval
lm_eval.tasks.TaskManager().initialize_tasks()
task_dict = lm_eval.tasks.get_task_dict('wikitext')
quanto.quantize(model=model, activations=quanto.qint8, weights=None)
moe =eval_wrapper(pretrained=model, tokenizer=tokenizer)
eval_results = lm_eval.evaluate(
        moe,
        task_dict,
    )

2024-04-05:10:07:10,271 WARNING  [task.py:763] [Task: wikitext] metric word_perplexity is defined, but aggregation is not. using default aggregation=weighted_perplexity
2024-04-05:10:07:10,274 WARNING  [task.py:775] [Task: wikitext] metric word_perplexity is defined, but higher_is_better is not. using default higher_is_better=False
2024-04-05:10:07:10,275 WARNING  [task.py:763] [Task: wikitext] metric byte_perplexity is defined, but aggregation is not. using default aggregation=weighted_perplexity
2024-04-05:10:07:10,276 WARNING  [task.py:775] [Task: wikitext] metric byte_perplexity is defined, but higher_is_better is not. using default higher_is_better=False
2024-04-05:10:07:10,277 WARNING  [task.py:763] [Task: wikitext] metric bits_per_byte is defined, but aggregation is not. using default aggregation=bits_per_byte
2024-04-05:10:07:10,278 WARNING  [task.py:775] [Task: wikitext] metric bits_per_byte is defined, but higher_is_better is not. using default higher_is_better=False
2024-04-

TypeError: linear(): argument 'weight' (position 2) must be Tensor, not NoneType